In [6]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import os
# widely used for a variety of image and video analysis
import cv2
import random
import glob

In [2]:
df = pd.read_csv('/Users/cochral/Desktop/SLAEP/EXTRACT/FOOD-2%/DAY4/N10/2024-04-30_14-31-44_td5.000_2024-04-30_14-31-44_td5.analysis.csv')

In [5]:
# Set the image size
image_size = 1400
# np.zeros creates a new array filled with zeros
# creates a 1400x1400 pixel image with 3 color channels (RGB)
tracks_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

# ensure sorted by frame index
df = df.sort_values('frame_idx')

# Define a list of 10 colors for the tracks
def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))



# Create a dictionary to hold colors for each track
track_colors = {}

# Draw the tracks for each unique track ID
  # ind = index of current iteration
    # required an index (enumerate) such that each loop through, for each distinct track, will be assigned a new color 
  # track = each unique track e.g. track 9 
for ind, track in enumerate(df['track'].unique()):
    
    
    track_df = df[df['track'] == track]

    track_colors[track] = generate_random_color()

    # Loop through the color palette and assign the track a new color 


    # Iterate through the track data and draw lines between consecutive points
       # the use of range(len()) will create a sequence of numbers - how to loop over the index 
       # the for loop will run from the second element (index 1) to the last element in the dataframe
       # the reason for starting at index 1 is lines will be drawn by comparing the coordinate with its predecessor to draw a line between them
    for i in range(1, len(track_df)):
        
        # Get the previous and current frame data
          # define the current and previous index 
          # will be used to compare the two rows and draw appropriate lines 
          # iloc = integer index-based -  have to specify rows and columns by their integer index (loc = specify name)
        prev_frame = track_df.iloc[i-1]
        curr_frame = track_df.iloc[i]

        # Check if the current or previous frame contains NaN in any of the required columns
        if not prev_frame[['head.x', 'head.y', 'body.x', 'body.y', 'tail.x', 'tail.y']].isna().any() and not curr_frame[['head.x', 'head.y', 'body.x', 'body.y', 'tail.x', 'tail.y']].isna().any():
            
            # Draw lines between the corresponding head, body, and tail points
              # must be an integer for cv2
              # cv2.line(img, pt1, pt2, color, thickness=None, lineType=None, shift=None)
            cv2.line(tracks_image, 
                     (int(prev_frame['head.x']), int(prev_frame['head.y'])), 
                     (int(curr_frame['head.x']), int(curr_frame['head.y'])), 
                     color=track_colors[track], 
                     thickness=1)
            cv2.line(tracks_image, 
                     (int(prev_frame['body.x']), int(prev_frame['body.y'])), 
                     (int(curr_frame['body.x']), int(curr_frame['body.y'])), 
                     color=track_colors[track], 
                     thickness=1)
            cv2.line(tracks_image, 
                     (int(prev_frame['tail.x']), int(prev_frame['tail.y'])), 
                     (int(curr_frame['tail.x']), int(curr_frame['tail.y'])), 
                     color=track_colors[track], 
                     thickness=1)

# Save the image
cv2.imwrite('/Users/cochral/repos/behavioural-analysis/plots/attraction-rig/new.png', tracks_image)

True

In [15]:
import cv2
import pandas as pd
import random
import os
import glob

# Function to generate a random color
def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Set the image size
image_size = 1400

# Directory containing the CSV files
directory_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/developmental-agarose/day5-n10'
output_directory = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/developmental-agarose/day5-n10'

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Loop through all CSV files in the directory
for csv_file in glob.glob(os.path.join(directory_path, '*.csv')):
    # Load the dataframe from a csv file
    df = pd.read_csv(csv_file)
    df = df.sort_values('frame_idx')

    # Create an image array
    tracks_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

    # Create a dictionary to hold colors for each track
    track_colors = {}

    # Process each track
    for track in df['track'].unique():
        track_df = df[df['track'] == track]
        track_colors[track] = generate_random_color()

        for i in range(1, len(track_df)):
            prev_frame = track_df.iloc[i-1]
            curr_frame = track_df.iloc[i]

            if not prev_frame[['head.x', 'head.y', 'body.x', 'body.y', 'tail.x', 'tail.y']].isna().any() and not curr_frame[['head.x', 'head.y', 'body.x', 'body.y', 'tail.x', 'tail.y']].isna().any():
                # Draw lines for each segment
                cv2.line(tracks_image, 
                         (int(prev_frame['head.x']), int(prev_frame['head.y'])), 
                         (int(curr_frame['head.x']), int(curr_frame['head.y'])), 
                         color=track_colors[track], 
                         thickness=1)
                cv2.line(tracks_image, 
                         (int(prev_frame['body.x']), int(prev_frame['body.y'])), 
                         (int(curr_frame['body.x']), int(curr_frame['body.y'])), 
                         color=track_colors[track], 
                         thickness=1)
                cv2.line(tracks_image, 
                         (int(prev_frame['tail.x']), int(prev_frame['tail.y'])), 
                         (int(curr_frame['tail.x']), int(curr_frame['tail.y'])), 
                         color=track_colors[track], 
                         thickness=1)

    # Construct the output file name based on the CSV file name
    output_file_name = os.path.basename(csv_file).replace('.csv', '.png')
    output_file_path = os.path.join(output_directory, output_file_name)

    # Save the image
    cv2.imwrite(output_file_path, tracks_image)
